# Ifremer prototype: balast

Balasting is a two step procedures:

1. in a controled environment, the *volume* $V$ of the float must be measured
2. pre-deployment, the mass of the float needs to be adjusted in order to adjust to local conditions

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import gsw

import cognac.ufloat as fl

----

## step 1: computation of the volume in a controled environment

In order to estimate the volume of the float, one must weigh it in air (mass $m$) and under water (mass $m^{water}$).
The target precision is high (typically gramms) and this can only be done in a controled environment (still water, known temperature, salinity) with accurate tools (temperature, salinity sensors, balance).
Precisions required for this instruments will be specified next.

The mass of the float underwater is given by:

$
\begin{align}
m^{water} &=  (\rho-\rho_{water})V \\
&= m - \rho_{water} V,
\end{align}
$

where $\rho = m/V$ is the float density.
This leads to:

$
\begin{align}
V &= \frac{m-m^{water}}{\rho_{water}} 
\end{align}
$


An additional mass ($m_{add}^{water}$ measured underwater) may have to be added in order to submerge the float.
The mass of the float underwater is then related to the total weight under water $m_{float+add}^{water}$ by:

$
\begin{align}
m^{water} &= m_{float+add}^{water} - m_{add}^{water}
\end{align}
$

Volume uncertainties associated with unaccurate masses, temperature, salinity are given by:

$
\begin{align}
e_{m} &= \delta m /\rho_{water}, \\
e_{T} &= V \times \frac{\partial_T\rho_{water}}{\rho_{water}}  \delta T = V \times \alpha \delta T, \\
e_{S} &= V \times \frac{\partial_S\rho_{water}}{\rho_{water}}  \delta S = V \times \beta \delta S.
\end{align}
$

For $\delta_m = 1$g, $\delta T=0.1$degC, $\delta S = 0.1$psu, $V=12\times 10^3$cm^3 (see cell below):

$
\begin{align}
e_{m} &\sim 1.0\;cm^3 \\
e_{T} &\sim 0.3\;cm^3, \\
e_{S} &\sim 0.9\;cm^3.
\end{align}
$

In [8]:
# select a water profile
w = fl.waterp(lon=6.,lat=40.) # Mediterranean sea
z=np.arange(-100.,0.,1.)

gamma = w.get_compressibility(z).mean()
alpha = w.get_alpha(z).mean()
beta = w.get_beta(z).mean()

print('gamma_water = {:.3} /dbar'.format(gamma) )
print('alpha_water = {:.3} /degC'.format(alpha) )
print('beta_water = {:.3} /psu'.format(beta) )

V = 12*1e3 # cm^3
dm = 1 # g
dT = .1 # degC
dS = .1 # psu
rho_water = 1030 # kg

print('e_m = {:.3} cm3'.format(dm*1e-3/rho_water*1e6) )
print('e_T = {:.3} cm3'.format(V*alpha*dT) )
print('e_S = {:.3} cm3'.format(V*beta*dS) )

gamma_water = 4.24e-06 /dbar
alpha_water = 0.000231 /degC
beta_water = 0.000738 /psu
e_m = 0.971 cm3
e_T = 0.277 cm3
e_S = 0.886 cm3


---

## step 2: adjustment of the mass predeployment

We compute the mass adjustement required with the following equations.

$
\begin{align}
\delta m^{water} = 0 &= \delta m - V \delta \rho_{water} - \rho_{water} \delta V,
\end{align}
$

$\delta m$ is the mass that needs to be added or substracted and is therefore given by:

$
\begin{align}
\delta m  &= V \delta \rho_{water} + \rho_{water} \delta V,
\end{align}
$

Note that unlike step 1, this step takes into account the float thermal compressibility via $\delta V$.

Any uncertainties on water temperature/salinity will induce uncertainties comparable to that of step 1:

In [10]:
print('e_T = {:.3} g'.format(V*alpha*rho_water/1e3*dT) )
print('e_S = {:.3} g'.format(V*beta*rho_water/1e3*dS) )

e_T = 0.286 g
e_S = 0.913 g


---

## 09/2020 balastage cuve acoustique (step 1)

In [2]:
pressure = 1.
temperature_water = 22.57
# 23 - 997.541 kg/m3
# 22 - 997.774 kg/m3

CT = gsw.CT_from_t(0., temperature_water, pressure)
rho_water = gsw.rho(0., CT, pressure)
rho_water

997.6474962446503

In [3]:
CT

23.737635299526843

In [4]:
m_float = 13.641 # mass float in air
m_float_add_water = .298 # mass float in water
m_add_water = 0. # additional mass in water

#V = (m_float + m_add_water - m_float_add_water)/(rho_water+.2) # 13371.8 cm^3 -> 2.7 cm^3 diff
V = (m_float + m_add_water - m_float_add_water)/rho_water # 13374.5 cm^3
V*1e6 # cm^3

13374.463475552024

In [5]:
f = fl.autonomous_float(model='Ifremer')
f

Float parameters: 
  L     = 0.83 m      - float length
  r     = 0.07 m      - float radius
  m     = 11.63 kg     - float mass
  V     = 1.27e+04 cm^3   - float volume
  rho_cte = m/V = 9.13e+08 kg.cm^3   - float baseline density
  gamma = 3.78e-06 /dbar  - mechanical compressibility
  gamma x V = 4.82e-02 cm^3/dbar  - normalized compressibility
  alpha = 6.98e-05 /degC  - thermal compressibility
  alpha x V = 8.89e-01 cm^3/degC  - normalized thermal compressibility
  temp0 = 0.00e+00  degC  - reference temperature
  a = 1.00e+00  (no dimension)  - float added mass
  c0 = 0.00e+00  (no dimension)  - float drag parameter 0
  c1 = 1.00e+00  (no dimension)  - float drag parameter 1

---
## 09/2020 cuve acoustique puis bassin

In [6]:
bfile = '/Users/aponte/Current_projects/cognac/tests/202009_ifremer/balast.yaml'

In [4]:
b = fl.balast()
b.add_balast('cuve_acoustique-20/09/2020', 
             mass_in_water=298,
             mass_in_air=13641., 
             water_temperature=22.57,
             water_salinity=0.,
             piston_displacement='max',
            )
#
b.store(bfile)

Balasting data stored in /Users/aponte/Current_projects/cognac/tests/202009_ifremer/balast.yaml


In [8]:
# reload
b = fl.balast(bfile)

File /Users/aponte/Current_projects/cognac/tests/202009_ifremer/balast.yaml loaded


In [9]:
b

{ 'cuve_acoustique-20/09/2020': { 'CT': 23.737635299526843,
                                  'SA': 0.0,
                                  'V': 0.013374463475552024,
                                  'comments': 'None',
                                  'mass_in_air': 13641.0,
                                  'mass_in_water': 298,
                                  'piston_displacement': 'max',
                                  'rho_water': 997.6474962446503,
                                  'water_salinity': 0.0,
                                  'water_temperature': 22.57}}

In [10]:
# init float
f = fl.autonomous_float(model='Ifremer')
f.init_piston()

Piston max volume set from max displacement


In [20]:
# à 1m: 35.1 psu, 19.0 degC
# uncertainty of .1 psu (1g) and .1 degC (0.2g)
b.compute_mass_adjustment(f, 
                          temperature=19.0, 
                          salinity=35.1, 
                          lon='4W30.', 
                          lat='48N00.'
                         )

-- According to balast cuve_acoustique-20/09/2020, you need add 65.7 g
(if the weight is external to the float, remember this must be the value in water)


In [21]:
add = 158 # 156, 158, 160 in water
float_plus_add = 94
add - float_plus_add

64

---

### bassin

In [23]:
# à 1m: 36.6 psu, 16.7 degC
# à la surface: 36.7 17 degC
b.compute_mass_adjustment(f, 
                          temperature=16.7, 
                          salinity=36.6, 
                          lon='4W30.', 
                          lat='48N00.'
                         )

-- According to balast cuve_acoustique-20/09/2020, you need add 86.4 g
(if the weight is external to the float, remember this must be the value in water)


In [17]:
94-86.4

7.599999999999994

In [19]:
m_float_add = 46 # water
m_add = 130 # water
m_float = m_float_add - m_add
m_float

-84

In [21]:
m_float_add = 212 # water
m_add = 286 # water
m_float = m_float_add - m_add
m_float

-74

In [15]:
f.v

2.6878288646869176e-05

---

## first deployment: lopstech19 leg 2

In [2]:

bfile = '/Users/aponte/Current_projects/cognac/campagnes_techno/2019_lopstech/leg2/float/balast.yaml'

### generate balast file

In [2]:
#
b = fl.balast()

In [3]:
b.add_balast('ifremer-15/10/2019', mass_in_water=0., mass_in_air=13554., 
             water_temperature=17.4, water_salinity=34.622, 
             piston_displacement=9.*3./4.)
#
b.store(bfile)

Balasting data stored in /Users/aponte/Current_projects/cognac/campagnes_techno/2019_lopstech/leg2/float/balast.yaml


### reload balast file

In [3]:
b = fl.balast(file=bfile)

File /Users/aponte/Current_projects/cognac/campagnes_techno/2019_lopstech/leg2/float/balast.yaml loaded


In [4]:
b

{ 'ifremer-15/10/2019': { 'CT': 17.40100549537287,
                          'SA': 34.78738585389333,
                          'V': 0.013221626276530922,
                          'comments': 'None',
                          'mass_in_air': 13554.0,
                          'mass_in_water': 0.0,
                          'piston_displacement': 6.75,
                          'rho_water': 1025.1386415345182,
                          'water_salinity': 34.622,
                          'water_temperature': 17.4}}

In [5]:
# init float
f = fl.autonomous_float(model='Ifremer')
f.init_piston()

Piston max volume set from max displacement


---

### compute adjustments

In [18]:
# use 1 point to compute mass adjustment
# could think also about loading tsg data
#b.compute_mass_adjustment(f, temperature=19.4, salinity=37.99, lon='5E23.634', lat='43N02.757')
#b.compute_mass_adjustment(f, temperature=19.97, salinity=37.82, lon='5E26.758', lat='43N00.910')
b.compute_mass_adjustment(f, temperature=19.824, salinity=37.85, lon='5E29.328', lat='43N00.390') # d0
# with this last balast, the float was sinking at nbpulse_gpiston (14??). 
#    but with 20g off, it was floating.
b.compute_mass_adjustment(f, temperature=18.643, salinity=38.021, lon='5E34.638', lat='42N53.959') # d1

According to balast ifremer-15/10/2019, you need add 43.3 g
According to balast ifremer-15/10/2019, you need add 48.0 g


In [ ]:
# use water profile to compute mass adjustement
w = fl.waterp(lon=6.,lat=40.) # Mediterranean sea
print(w)
b.compute_mass_adjustment(f, w)

---

# 2019/11: tests ifremer

In [2]:
bfile = '/Users/aponte/Current_projects/cognac/instrum/flotteur/tests/balast_201911.yaml'

In [3]:
#
b = fl.balast()

In [4]:
b.add_balast('ifremer-18/11/2019', mass_in_water=-121., mass_in_air=13413., 
             water_temperature=18.2706, water_conductivity=44.6036, 
             piston_displacement=0.)
#
b.store(bfile)

Practical salinity derived from conductivity: SP=33.67
Balasting data stored in /Users/aponte/Current_projects/cognac/instrum/flotteur/tests/balast_201911.yaml


In [3]:
b = fl.balast(file=bfile)

File /Users/aponte/Current_projects/cognac/instrum/flotteur/tests/balast_201911.yaml loaded


In [4]:
b

{ 'ifremer-18/11/2019': { 'CT': 18.300396039190872,
                          'SA': 33.83300763196808,
                          'V': 0.01309608458116599,
                          'comments': 'None',
                          'mass_in_air': 13413.0,
                          'mass_in_water': -121.0,
                          'piston_displacement': 0.0,
                          'rho_water': 1024.199249544385,
                          'water_salinity': 33.672159073801225,
                          'water_temperature': 18.2706}}

In [5]:
# init float
f = fl.autonomous_float(model='Ifremer')
f.init_piston()

Piston max volume set from max displacement


In [6]:
# use 1 point to compute mass adjustment
b.compute_mass_adjustment(f, temperature=12.99, conductivity=42.5, 
                          lon=b.lon, lat=b.lat, verbose=True)

-- According to balast ifremer-18/11/2019, you need add 159.9 g
(if the weight is external to the float, remember this must be the value in water)
Independent mass correction (balast -mass_in_water): 121.0 [g]
  Water change mass correction: 38.9 [g]
    T contribution: 14.3 [g]
    S contribution: 28.2 [g]
  New/old in situ density: 1027.55, 1024.20  [kg/m^3] 
    difference: 3.4 [kg/m^3]
  New/old float volume: 1.31e-02, 1.31e-02  [m^3] 
    difference: -4.8 [cm^3]


**Observed behavior**:
- with 117.5g (in water) the float does not sink even with the piston fully in (+30g approx)
- with 128.5g (in water) the float is able to sink with the piston fully in (+30g approx)

---
## other interesting values

In [13]:
w = fl.waterp(lon=6.,lat=40.) # Mediterranean sea

print('Float thermal expansion = %1.2e' %(f.alpha))
alpha, beta = w.get_alpha(-1.), w.get_beta(-1.)
print('Water: thermal expansion = %1.2e and haline contraction =% 1.2e' %(alpha, beta))

dT, dS = 1., 1.
for name, _b in b._d.items():
    print('%.1f deg float temperature change leads to an equivalent mass change of %.2f g' %(dT, f.alpha*f.m*dT*1e3))
    print('%.1f deg water temperature change leads to an equivalent mass change of %.2f g' %(dT,alpha*f.m*dT*1e3))
    print('%.1f g/kg salinity change leads to an equivalent mass change of %.2f g' %(dS,beta*f.m*dS*1e3))

Float thermal expansion = 6.98e-05
Water: thermal expansion = 2.57e-04 and haline contraction = 7.33e-04
1.0 deg float temperature change leads to an equivalent mass change of 0.94 g
1.0 deg water temperature change leads to an equivalent mass change of 3.45 g
1.0 g/kg salinity change leads to an equivalent mass change of 9.83 g


In [12]:
f

Float parameters: 
  L     = 0.83 m      - float length
  r     = 0.07 m      - float radius
  m     = 13.41 kg     - float mass
  V     = 1.31e+04 cm^3   - float volume
  rho_cte = m/V = 9.13e+08 kg.cm^3   - float baseline density
  gamma = 3.78e-06 /dbar  - mechanical compressibility
  alpha = 6.98e-05 /degC  - thermal compressibility
  temp0 = 1.83e+01  degC  - reference temperature
  a = 1.00e+00  (no dimension)  - float added mass
  c0 = 0.00e+00  (no dimension)  - float drag parameter 0
  c1 = 1.00e+00  (no dimension)  - float drag parameter 1
Piston parameters and state: 
  r     = 0.97 cm        - piston radius
  d     = 0.00 cm        - present piston displacement
  vol   = 0.00 cm^3      - present volume addition
  d_increment  = 2.14e-02 mm        - smallest variation of translation motion for the piston
  vol_increment  = 6.40e-03 cm^3        - smallest variation of volume possible for the piston
  d_max = 90.00 mm        - maximum piston displacement
  d_min = 0.00 mm     

---

### lead sheets weight in water / weight in air:

In [10]:
print(30/41)
print(19/25)
print(20/28)

0.7317073170731707
0.76
0.7142857142857143
